## Group Assignment

#### Import the necessary libraries

In [126]:
import numpy as np
import pandas as pd

import datetime as dt
import yfinance as yf
import pandas_datareader.data as web

import matplotlib.pyplot as plt
%matplotlib inline

#### All questions in Parts I and II apply to a random sample of 15 stocks that your group will be assigned by running the following code.

#### Random Sample Selection 

In [127]:
np.random.seed (2051 + 6)
ticker_list = ['AAPL', 'AXP', 'BA', 'C','CAT', 'CSCO', 'CVX', 'DIS', 'GS', 'HD', 'IBM', 'INTC', 'JNJ', 'JPM', 'KO',
            'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PFE', 'PG', 'TRV', 'UNH', 'V', 'VZ', 'WMT', 'XOM']

stock_list = np.random.choice(ticker_list,15,replace=False)
print(f'These are the fifteen stocks assigned to you: {" ".join(stock_list)}')

These are the fifteen stocks assigned to you: DIS PG V UNH CSCO PFE JPM AXP JNJ INTC KO CAT WMT CVX XOM


### PART I

#### 1. Find the optimal portfolio over the period January 2015 - December 2019, using the fifteen stocks assigned to your group. Assume there are no short-selling constraints.

In [128]:
start = dt.datetime(2015,1,1)
end = dt.datetime(2019,12,31)

stock_list=['DIS', 'PG', 'V', 'UNH', 'CSCO', 'PFE', 'JPM', 'AXP', 'JNJ', 'INTC', 'KO', 'CAT', 'WMT', 'CVX', 'XOM']

returns = yf.download(stock_list,start-pd.offsets.BDay(1),end+pd.offsets.BDay(1))['Adj Close'].pct_change().dropna()

[*********************100%***********************]  15 of 15 completed


In [129]:
from scipy.optimize import minimize

In [130]:
returns_mon = returns.resample('M').apply(lambda x: x.add(1).prod().sub(1))

In [131]:
rf = web.DataReader('F-F_Research_Data_Factors','famafrench', start, end)[0][['RF']].div(100)
rf.index = rf.index.to_timestamp(how='end').normalize()

In [132]:
def port_ret(weights):
    port_ret = np.dot(returns_mon*12,weights).mean()
    return port_ret

def port_std(weights):
    port_std = np.sqrt(np.dot(weights, np.dot(returns_mon.cov()*12, weights)))
    return port_std

def ex_port_ret(weights):
    ex_port_ret = (np.dot(returns_mon,weights) - rf['RF']).mean()*12
    return ex_port_ret

def ex_port_std(weights):
    ex_port_std = (np.dot(returns_mon,weights) - rf['RF']).std()*np.sqrt(12)
    return ex_port_std

def neg_SR(weights):
    SR = ex_port_ret(weights) / ex_port_std(weights)
    return (-1)*SR

In [133]:
constraints = ({'type':'eq','fun': lambda weights: np.sum(weights) - 1})

boundaries=[(0,1)]
bounds = tuple(boundaries * len(returns_mon.columns))

init_guess = np.full(len(returns_mon.columns), 1/len(returns_mon.columns))

In [134]:
optimal_port=minimize(neg_SR,init_guess,constraints=constraints)
optimal_port

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: -2.3196550603957355
       x: [-4.877e-01 -1.138e-03 ...  4.103e-02 -5.622e-01]
     nit: 16
     jac: [-5.651e-02 -5.624e-02 ... -5.595e-02 -5.633e-02]
    nfev: 259
    njev: 16

#### 2. What are the weights of the stocks in the optimal portfolio?

In [135]:
for ticker in stock_list:
    result=print(ticker + "  "+ str(round(optimal_port.x[stock_list.index(ticker)],4)))

DIS  -0.4877
PG  -0.0011
V  0.0228
UNH  0.2157
CSCO  -0.1961
PFE  0.0588
JPM  0.0261
AXP  0.5186
JNJ  -0.1167
INTC  -0.2932
KO  0.324
CAT  0.5926
WMT  0.8573
CVX  0.041
XOM  -0.5622


#### 3. What was the annualized average monthly return for the optimal portfolio?

In [136]:
optimal_weights = optimal_port.x
optimal_annual_return = port_ret(optimal_weights) * 12

#### 4. What was the annualized monthly standard deviation for the optimal portfolio?

In [137]:
optimal_std=port_std(optimal_weights) * 12

#### 5. What was the Sharpe Ratio of the optimal portfolio?

In [138]:
-optimal_port.fun

2.3196550603957355

### PART II

#### 1. Create a DataFrame named optimal_weights to store the weights from a portfolio optimization performed on a rolling basis. Specifically, the optimization should use a 60 month rolling window, and be performed every month. The first 60 months correspond to the sample period for data_initial (01/2015 - 12/2019) created in question 3. Hence, the first observation in the optimal_weights DataFrame should be the weights you listed above in Part I. The next 60 month period should be 02/2015 - 01/2020, followed by 03/2015 - 02/2020, and so on. The last 60 month period should be 03/2018 - 02/2023. Again, assume there are no short-selling constraints.

In [139]:
start = dt.datetime(2015,1,1)
end = dt.datetime(2023,2,28)

stock_list=['DIS', 'PG', 'V', 'UNH', 'CSCO', 'PFE', 'JPM', 'AXP', 'JNJ', 'INTC', 'KO', 'CAT', 'WMT', 'CVX', 'XOM']

returns = yf.download(stock_list,start-pd.offsets.BDay(1),end+pd.offsets.BDay(1))['Adj Close'].pct_change().dropna()

[*********************100%***********************]  15 of 15 completed


In [140]:
rf_rate = web.DataReader('F-F_Research_Data_Factors','famafrench', start, end)[0][['RF']].div(100)
rf_rate

,RF
Date,
2015-01,0.0000
2015-02,0.0000
2015-03,0.0000
2015-04,0.0000
2015-05,0.0000
...,...
2022-10,0.0023
2022-11,0.0029
2022-12,0.0033


In [141]:
returns_mon1 = returns.resample('M').apply(lambda x: x.add(1).prod().sub(1))

In [142]:
optimal_weights = pd.DataFrame()

for i in range(0,len(returns_mon1.index)-60+1):

    def neg_SR(weights):
        SR = ((np.dot(returns_mon1.iloc[i:i+60],weights) - rf_rate.iloc[i:i+60]['RF']).mean()*(12)) /\
        ((np.dot(returns_mon1.iloc[i:i+60],weights) - rf_rate.iloc[i:i+60]['RF']).std()*np.sqrt(12))
        return (-1)*SR
    
    constraints = ({'type':'eq','fun': lambda weights: np.sum(weights) - 1})

    init_guess = np.full(len(returns_mon1.columns),1/len(returns_mon1.columns))
    
    optimal_port = minimize(neg_SR,init_guess,constraints = constraints)
    
    optimal_weights = pd.concat([optimal_weights,
                                pd.DataFrame(optimal_port.x.reshape(1,len(stock_list)).round(4),
                                             columns=[stock_list],
                                             index=[returns_mon1.iloc[i:i+60].index[-1]])],
                                axis=0)
optimal_weights.index.name='Date'
optimal_weights=optimal_weights.sort_index(axis=1)

#### 2. Create a new DataFrame named port_returns to store the monthly returns over the sample period 01/2020 - 02/2023 for the following portfolios:
 1) A monthly rebalanced portfolio using the rolling optimal weights. Specifically, the portfolio return for 01/2020 should be based on the weights as of 12/2019 (the same values detailed in Part I and in the first row in the optimal_weights DataFrame), the portfolio return for 02/2020 should be based on the weights as of 01/2020 and so on. Label the portfolio (MRoll_Reb_OP, an acronym for Monthly Rolling Rebalanced Optimal Portfolio). 
 2) A monthly rebalanced portfolio using the optimal weights as of 12/2019. Label the portfolio (M_Reb_OP, an acronym for Monthly Rebalancing Optimal Portfolio).
 3) A monthly rebalanced equally-weighted portfolio. Label the portfolio (M_Reb_EW, an acronym for Monthly Rebalancing Equally-Weighted Portfolio).
 4) A buy and hold portfolio, initially allocated according to the optimal weights as of 12/2019. Label the portfolio (BH_OP, an acronym for Buy and Hold Optimal Portfolio).
 5) A buy and hold portfolio, initially allocated equally across stocks. Label the portfolio (BH_EW, an acronym for Buy and Hold Equally-Weighted Portfolio).

In [143]:
inv_value = pd.DataFrame()
for column in returns_mon1:
    inv_value[column] = returns_mon1[column].div(returns_mon1[column].iloc[0])

In [144]:
init_guess = np.full(len(returns_mon1.columns), 1/len(returns_mon1.columns))

In [145]:
#1
MRoll_Reb_OP=pd.DataFrame()
for i in range(0,len(optimal_weights)-1):
    MRoll_Reb_OP = pd.concat([MRoll_Reb_OP,
                              pd.DataFrame({'MRoll_Reb_OP':np.dot(optimal_weights.iloc[i],returns_mon1.iloc[i+60])},
                                           index = [returns_mon1.index[i+60]])],
                                           axis=0)

In [146]:
MRoll_Reb_OP

,MRoll_Reb_OP
2020-01-31,-0.006183
2020-02-29,-0.006008
2020-03-31,0.091422
2020-04-30,-0.126004
2020-05-31,-0.009472
2020-06-30,0.023594
2020-07-31,0.267196
2020-08-31,0.163181
2020-09-30,0.121241
2020-10-31,0.052853


In [147]:
#2
M_Reb_OP = pd.DataFrame({'M_Reb_OP':np.dot(returns_mon1.iloc[0+60:], optimal_weights.iloc[0])},index = returns_mon1.index[0+60:])

In [148]:
M_Reb_OP

,M_Reb_OP
Date,
2020-01-31,-0.006183
2020-02-29,0.016789
2020-03-31,0.068884
2020-04-30,-0.070806
2020-05-31,-0.000514
2020-06-30,0.033679
2020-07-31,0.189920
2020-08-31,0.127391
2020-09-30,0.139851


In [149]:
#3
M_Reb_EW=pd.DataFrame()
for i in range(0,len(optimal_weights)-1):
    M_Reb_EW = pd.concat([M_Reb_EW,
                              pd.DataFrame({'MRoll_Reb_OP':np.dot(init_guess,returns_mon1.iloc[i+60])},
                                           index = [returns_mon1.index[i+60]])],
                                           axis=0)

In [150]:
M_Reb_EW

,MRoll_Reb_OP
2020-01-31,-0.022932
2020-02-29,-0.106329
2020-03-31,-0.102047
2020-04-30,0.117023
2020-05-31,0.035556
2020-06-30,-0.028010
2020-07-31,0.020298
2020-08-31,0.043138
2020-09-30,-0.032859
2020-10-31,-0.039936


In [151]:
#4
BH_OP = pd.DataFrame({'BH_OP':np.dot(inv_value, optimal_weights.iloc[0])},index = inv_value.index).loc['2019-12-31':'2023'].pct_change().dropna()

In [152]:
BH_OP

,BH_OP
Date,
2020-01-31,-0.813081
2020-02-29,3.215779
2020-03-31,42.281786
2020-04-30,-1.318990
2020-05-31,-1.220523
2020-06-30,0.990818
2020-07-31,3.286347
2020-08-31,0.498840
2020-09-30,-0.405328


In [153]:
#5
BH_EW = pd.DataFrame({'BH_EW':np.dot(inv_value, init_guess)},index = inv_value.index).loc['2019-12-31':'2023'].pct_change().dropna()

In [154]:
port_returns = pd.concat([MRoll_Reb_OP, M_Reb_OP, M_Reb_EW,BH_OP, BH_EW], axis=1)
port_returns

,MRoll_Reb_OP,M_Reb_OP,MRoll_Reb_OP,BH_OP,BH_EW
2020-01-31,-0.006183,-0.006183,-0.022932,-0.813081,-85.591761
2020-02-29,-0.006008,0.016789,-0.106329,3.215779,-0.794301
2020-03-31,0.091422,0.068884,-0.102047,42.281786,-8.821476
2020-04-30,-0.126004,-0.070806,0.117023,-1.318990,0.667018
2020-05-31,-0.009472,-0.000514,0.035556,-1.220523,-1.395482
2020-06-30,0.023594,0.033679,-0.028010,0.990818,2.522627
2020-07-31,0.267196,0.189920,0.020298,3.286347,-2.339040
2020-08-31,0.163181,0.127391,0.043138,0.498840,-1.492519
2020-09-30,0.121241,0.139851,-0.032859,-0.405328,-1.001868
2020-10-31,0.052853,0.072092,-0.039936,-0.634679,35.868562


#### 3. Create a DataFrame port_stats (with a 3x5 shape) to store the annualized average monthly return, the annualized monthly standard deviation and the Sharpe ratio for all 5 portfolios.

In [ ]:
port_stats=pd.DataFrame()


#### 4. Compute the cumulative return series for all portfolios and plot them in the same graph. 

#### 5. What was the cumulative return for the best performing portfolio? What was the cumulative return for the worst performing portfolio?

### PART III - Examination of the Size Effect

Please download the file "crsp_fall22.csv" located in folder Group_Assignment. The file contains the following columns:
* PERMNO - Permanent number (unique identifer for the stock)
* DATE
* PRC - The closing price
* VOL - Trading volume (in hundreds)
* RET - The simple return
* SHROUT - The number of shares outstanding (in thousands)
* CFACPR - Cumulative factor to adjust the closing price
* CFACSHR - Cumulative factor to adjust shares outstanding

#### 1. Create a DataFrame named fin_data by reading in the columns DATE, PERMNO, RET, PRC and SHROUT from the file 'crsp_fall22.csv'. Set the DATE as the index. In addition, change each index value to the end of the month. For example, change '1926-05-30' to 1926-05-31', or '1987-01-30' to '1987-01-31'. In addition, create a new column (labeled MKTCAP) to store the market capitalization (defined as PRC * SHROUT). Subsequently, permanently remove the PRC and SHROUT columns.

In [368]:
fin_data=pd.read_csv('crsp_fall22.csv',
                     usecols=['DATE','PERMNO','RET','PRC','SHROUT'],
                     dtype={'PERMNO':np.int64},
                     index_col='DATE',
                     parse_dates=True)
fin_data.index=fin_data.index+pd.offsets.MonthEnd(0)
fin_data['MKTCAP']=fin_data['PRC']*fin_data['SHROUT']
fin_data.drop(['PRC','SHROUT'],axis=1,inplace=True)

#### 2. How many stocks are there in the sample? How many of these stocks are in the sample over the entire sample period?

In [17]:
fin_data.nunique()


PERMNO      18153
RET        828585
MKTCAP    1611452
dtype: int64

#### 3. Every June, split the stocks into quintiles (five groups) based on their market capitalization. The header of a DataFrame named size_qt with the resulting split can be found below. 

In [101]:
size_qt.sort_index().head()
#use pd.qcut

,PERMNO,MKTCAP_QT
DATE,,
1990-06-30,10294,3
1990-06-30,65496,3
1990-06-30,10905,2
1990-06-30,32037,1
1990-06-30,66288,5


#### 4. Create a new DataFrame named data by using the merge_asof() function to merge the DataFrames fin_data and size_qt. Specifically, merge each stock's June quintile allocation with the stock's monthly returns for the next 12 months starting the following July. Subsequently, set DATE as the index in the DataFrame.

In [ ]:
data=pd.DataFrame()

#### 5. Create a new DataFrame called quintiles to store the average monthly return for each quintile. The header of the DataFrame can be found below:

In [105]:
quintiles.head()

,Q1_RET,Q2_RET,Q3_RET,Q4_RET,Q5_RET
DATE,,,,,
1990-07-31,-0.001078,-0.041629,-0.044858,-0.040050,-0.028895
1990-08-31,-0.088672,-0.111944,-0.134610,-0.132793,-0.110817
1990-09-30,-0.045567,-0.090744,-0.092048,-0.099917,-0.080715
1990-10-31,-0.071408,-0.064675,-0.069424,-0.060729,-0.039041
1990-11-30,-0.011988,0.002373,0.033031,0.070116,0.099023


#### 6. Create a bar plot of the average monthly return for all five market capitalization groups, across the entire sample.

#### 7. Create a line plot of the cumulative return series for all five market capitalization groups.